In [ ]:
### Short script to convert rotations from FEA models represented as quaternions to Euler angles using scipy
### aka baby's first real python script
### 7-22-20

In [109]:
import numpy as np
import pandas as pd
import scipy

In [110]:
from scipy import spatial

In [111]:
spatial.transform.Rotation?

In [112]:
#read in ROM data in quaternions from final timestep

quatern = pd.read_csv('t1_quatern.csv')

In [113]:
quatern

,species,fsu,load,qx,qy,qz,qw
0,bighorn,C2-C3,axial,0.011158,-0.001718,0.000842,0.999936
1,bighorn,C2-C3,compress,0.000000,0.000000,0.000000,1.000000
2,bighorn,C2-C3,extend,-0.005598,0.000037,-0.019395,0.999796
3,bighorn,C2-C3,flex,-0.000203,-0.000027,0.008181,0.999967
4,bighorn,C2-C3,lateral,-0.001138,0.010153,-0.001768,0.999946
5,bighorn,C4-C5,axial,0.019101,-0.000493,-0.001293,0.999817
6,bighorn,C4-C5,compress,0.000000,0.000000,0.000000,1.000000
7,bighorn,C4-C5,extend,-0.000907,0.001144,-0.026126,0.999658
8,bighorn,C4-C5,flex,-0.000257,-0.000205,0.009719,0.999953
9,bighorn,C4-C5,lateral,0.000001,0.010761,-0.003306,0.999937


In [114]:
#import rotation module

from scipy.spatial.transform import Rotation as R

In [115]:
# create hierarchical index for rotation data

index = pd.MultiIndex.from_product([['bighorn', 'impala'], ['C2-C3', 'C4-C5', 'C6-C7'], 
                                    ['axial', 'compress', 'extend', 'flex', 'lateral']], 
                                   names=['species', 'fsu', 'load'])

In [116]:
# select the quaternion matrix rows -- not sure I need to do this but what the hell
# practicing slicing dataframes 
quat = pd.DataFrame(quatern[['qx', 'qy', 'qz', 'qw']])

In [117]:
# reindex the dataframe
quat.index = index

In [118]:
# whoohoo it's indexed
quat

qx        qy        qz        qw
species fsu   load                                            
bighorn C2-C3 axial     0.011158 -0.001718  0.000842  0.999936
              compress  0.000000  0.000000  0.000000  1.000000
              extend   -0.005598  0.000037 -0.019395  0.999796
              flex     -0.000203 -0.000027  0.008181  0.999967
              lateral  -0.001138  0.010153 -0.001768  0.999946
        C4-C5 axial     0.019101 -0.000493 -0.001293  0.999817
              compress  0.000000  0.000000  0.000000  1.000000
              extend   -0.000907  0.001144 -0.026126  0.999658
              flex     -0.000257 -0.000205  0.009719  0.999953
              lateral   0.000001  0.010761 -0.003306  0.999937
        C6-C7 axial     0.033228 -0.002805 -0.004820  0.999432
              compress  0.000000  0.000000  0.000000  1.000000
              extend   -0.000462  0.001045 -0.034604  0.999400
              flex     -0.000435 -0.000237  0.011025  0.999939
              lateral  -0.002483  0.021190 -0.007560  0.999744
impala  C2-C3 axial     0.068609 -0.017038 -0.011768  0.997429
              compress  0.000000  0.000000  0.000000  1.000000
              extend    0.002413  0.005573 -0.066107  0.997794
              flex     -0.001189 -0.000735  0.021684  0.999764
              lateral  -0.013055  0.062568 -0.015224  0.997839
        C4-C5 axial     0.058998 -0.012981 -0.006342  0.998154
              compress  0.000000  0.000000  0.000000  1.000000
              extend    0.001279 -0.000629 -0.073094  0.997324
              flex     -0.003271  0.001137  0.019621  0.999801
              lateral  -0.001374  0.039334 -0.011692  0.999157
        C6-C7 axial     0.069273 -0.009544 -0.015341  0.997434
              compress  0.000000  0.000000  0.000000  1.000000
              extend    0.001780  0.001199 -0.076293  0.997083
              flex     -0.001255 -0.000107  0.017505  0.999846
              lateral  -0.007408  0.036550 -0.018641  0.999130

In [119]:
# pull out one row of the quat matrix, practice this infuriating indexing system
trial = quat.iloc[:, 0:4,].iloc[0:1]

In [120]:
#oh look on try number 30 I managed to index a whole row, woopdeedoo
trial

,,,qx,qy,qz,qw
species,fsu,load,,,,
bighorn,C2-C3,axial,0.011158,-0.001718,0.000842,0.999936


In [121]:
# read in as rotation data
trial_r = R.from_quat(trial) #woah this actually worked

In [122]:
# view as quaternions 
trial_r.as_quat()

array([[ 1.11579991e-02, -1.71787986e-03,  8.42298930e-04,
         9.99935917e-01]])

In [123]:
# view as euler angles
trial_r.as_euler('xyz', degrees = True)

array([[ 1.27847848, -0.19791928,  0.09431826]])

In [124]:
#try it with all rows from the data
all_r = R.from_quat(quat)

In [125]:
all_r.as_quat().shape #this also worked!

(30, 4)

In [126]:
# convert all the quaternions to euler angles
euler = all_r.as_euler('xyz', degrees = True)

In [127]:
# make dataframe with same index
euler_df = pd.DataFrame(euler, index = index, columns = ['xdeg', 'ydeg', 'zdeg'])

In [129]:
# merge with original quaternion dataframe
rom_quat_euler = pd.merge(quat, euler_df, on = ['species', 'fsu', 'load'])


In [130]:
rom_quat_euler

qx        qy        qz        qw      xdeg  \
species fsu   load                                                         
bighorn C2-C3 axial     0.011158 -0.001718  0.000842  0.999936  1.278478   
              compress  0.000000  0.000000  0.000000  1.000000  0.000000   
              extend   -0.005598  0.000037 -0.019395  0.999796 -0.641455   
              flex     -0.000203 -0.000027  0.008181  0.999967 -0.023259   
              lateral  -0.001138  0.010153 -0.001768  0.999946 -0.132458   
        C4-C5 axial     0.019101 -0.000493 -0.001293  0.999817  2.189029   
              compress  0.000000  0.000000  0.000000  1.000000  0.000000   
              extend   -0.000907  0.001144 -0.026126  0.999658 -0.107336   
              flex     -0.000257 -0.000205  0.009719  0.999953 -0.029730   
              lateral   0.000001  0.010761 -0.003306  0.999937 -0.003941   
        C6-C7 axial     0.033228 -0.002805 -0.004820  0.999432  3.809943   
              compress  0.000000  0.000000  0.000000  1.000000  0.000000   
              extend   -0.000462  0.001045 -0.034604  0.999400 -0.057109   
              flex     -0.000435 -0.000237  0.011025  0.999939 -0.050121   
              lateral  -0.002483  0.021190 -0.007560  0.999744 -0.303130   
impala  C2-C3 axial     0.068609 -0.017038 -0.011768  0.997429  7.893834   
              compress  0.000000  0.000000  0.000000  1.000000  0.000000   
              extend    0.002413  0.005573 -0.066107  0.997794  0.233687   
              flex     -0.001189 -0.000735  0.021684  0.999764 -0.138025   
              lateral  -0.013055  0.062568 -0.015224  0.997839 -1.614706   
        C4-C5 axial     0.058998 -0.012981 -0.006342  0.998154  6.775497   
              compress  0.000000  0.000000  0.000000  1.000000  0.000000   
              extend    0.001279 -0.000629 -0.073094  0.997324  0.151430   
              flex     -0.003271  0.001137  0.019621  0.999801 -0.372207   
              lateral  -0.001374  0.039334 -0.011692  0.999157 -0.210725   
        C6-C7 axial     0.069273 -0.009544 -0.015341  0.997434  7.961240   
              compress  0.000000  0.000000  0.000000  1.000000  0.000000   
              extend    0.001780  0.001199 -0.076293  0.997083  0.192933   
              flex     -0.001255 -0.000107  0.017505  0.999846 -0.144063   
              lateral  -0.007408  0.036550 -0.018641  0.999130 -0.928725   

                            ydeg      zdeg  
species fsu   load                          
bighorn C2-C3 axial    -0.197919  0.094318  
              compress  0.000000  0.000000  
              extend   -0.008155 -2.222621  
              flex     -0.002851  0.937474  
              lateral   1.163224 -0.203905  
        C4-C5 axial    -0.053706 -0.149237  
              compress  0.000000  0.000000  
              extend    0.128361 -2.994236  
              flex     -0.023180  1.113741  
              lateral   1.233160 -0.378916  
        C6-C7 axial    -0.302849 -0.562720  
              compress  0.000000  0.000000  
              extend    0.117886 -3.966193  
              flex     -0.026592  1.263409  
              lateral   2.426180 -0.872970  
impala  C2-C3 axial    -1.855215 -1.479974  
              compress  0.000000  0.000000  
              extend    0.655538 -7.579602  
              flex     -0.081210  2.485041  
              lateral   7.150064 -1.849085  
        C4-C5 axial    -1.442097 -0.813485  
              compress  0.000000  0.000000  
              extend   -0.061223 -8.383499  
              flex      0.137646  2.248090  
              lateral   4.506300 -1.349199  
        C6-C7 axial    -0.969101 -1.829805  
              compress  0.000000  0.000000  
              extend    0.152519 -8.750813  
              flex     -0.009783  2.006042  
              lateral   4.172603 -2.171489

In [131]:
#write to csv 
pd.DataFrame.to_csv(rom_quat_euler, 'quaternToAngles.csv', index = True)